In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import sklearn
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset=pd.read_csv(r'C:\Users\lenovo\Desktop\Final Mca Project\NewCode\DataSet\New folder\Reviews1.csv')

In [3]:
len(dataset)

7

In [4]:
dataset.columns

Index(['ID', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [5]:
dataset.Text

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
5    I got a wild hair for taffy and ordered this f...
6    I got a wild hair for taffy and ordered this f...
Name: Text, dtype: object

In [6]:
final=dataset.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(6, 10)

In [7]:
final

,ID,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...


In [8]:
final.Text

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
5    I got a wild hair for taffy and ordered this f...
Name: Text, dtype: object

In [9]:

rev1 = final['Text'].values[0]
print(rev1)


I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [10]:
# remove urls from text
rev1 = re.sub(r"http\S+", "", rev1)

In [11]:
# remove tags
soup = BeautifulSoup(rev1, 'lxml')
text = soup.get_text()
print(text)


I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [12]:
# importnt function changing the words won't --> will not
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
rev1 = decontracted(rev1)
print(rev1)


I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [14]:
# remove words with numbers
rev1 = re.sub("\S*\d\S*", "", rev1).strip()
print(rev1)

I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.


In [15]:
#remove spacial character
rev1 = re.sub('[^A-Za-z0-9]+', ' ', rev1)
print(rev1)

I have bought several of the Vitality canned dog food products and have found them all to be of good quality The product looks more like a stew than a processed meat and it smells better My Labrador is finicky and she appreciates this product better than most 


In [16]:
rev1.lower()

'i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most '

In [17]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


False

In [18]:
stopwords = set(stopwords.words('english')) 
print(stopwords)

{'no', "wouldn't", 'ourselves', 'during', 'further', 'any', "you'll", 'has', 'who', 'above', "you're", 'himself', 'most', 'some', 've', 'when', 'needn', "she's", 'were', 'through', 'its', 'which', 'have', 'an', "mustn't", "you've", 'his', 'more', 'yours', 'will', 'theirs', "couldn't", 'doesn', 'ours', 'didn', 'not', "doesn't", 'did', 'where', 'their', 'nor', "needn't", 'you', 'should', 'our', 'after', 'it', 'be', 'until', 'itself', 'to', 'over', 'of', 'yourself', 'as', 'for', 'myself', 'herself', 'both', 'ma', 're', 'but', 'i', 'your', "mightn't", "haven't", 't', 'is', 'each', 'm', "that'll", "don't", 'hasn', 'if', 'while', 'am', 'under', 'now', 'he', 'themselves', "shan't", 'off', 'him', 'being', 'won', 'mustn', 'my', 'into', 'how', 'why', 'the', 'between', 'down', 'had', 'do', 'having', 'in', 'again', 'then', 'and', 'them', 'whom', 'too', 'her', 'doing', 'out', 'once', 'same', "you'd", 's', 'don', 'o', 'wouldn', 'this', 'ain', "didn't", "shouldn't", 'about', 'll', 'isn', 'only', 'tha

In [19]:
word_tokens = word_tokenize(rev1) 
  
filtered_sentence = [w for w in word_tokens if not w in stopwords] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stopwords: 
        filtered_sentence.append(w) 
  
# print(word_tokens) 
print(filtered_sentence) 

['I', 'bought', 'several', 'Vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'The', 'product', 'looks', 'like', 'stew', 'processed', 'meat', 'smells', 'better', 'My', 'Labrador', 'finicky', 'appreciates', 'product', 'better']


In [20]:
#stemmer=PorterStemmer()

In [25]:
#stemmer.stem(filtered_sentence[16])


'process'

In [23]:
stemmer=PorterStemmer()
length = len(filtered_sentence) 
for i in range(length):   
    for w in filtered_sentence: 
       s=stemmer.stem(filtered_sentence[i])
    print(s)  

I
bought
sever
vital
can
dog
food
product
found
good
qualiti
the
product
look
like
stew
process
meat
smell
better
My
labrador
finicki
appreci
product
better


In [24]:
vect = CountVectorizer()

In [25]:
vect.fit(s)

ValueError: Iterable over raw text documents expected, string object received.